#Create the environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/ESoWC

/content/drive/My Drive/ESoWC


In [3]:
import pandas as pd
import xarray as xr

import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns

In [4]:
fn_land = 'Data/land_cover_data.nc'
fn_weather = 'Data/05_2019_weather_and_CO_for_model.nc'
fn_conc = 'Data/conc_dataset.nc'
fn_traffic = 'Data/emissions_traffic_hourly_merged.nc'

#Load datasets

##Land

In [5]:
# Open netCDF file 
land = xr.open_dataset(fn_land)
land

<xarray.Dataset>
Dimensions:       (lat: 33, lon: 33, time: 30)
Coordinates:
  * time          (time) datetime64[ns] 2019-05-01 2019-05-02 ... 2019-05-30
  * lon           (lon) float64 4.0 4.25 4.5 4.75 5.0 ... 11.25 11.5 11.75 12.0
  * lat           (lat) float64 43.0 43.25 43.5 43.75 ... 50.25 50.5 50.75 51.0
Data variables:
    height        (time, lat, lon) float32 ...
    built         (time, lat, lon) float32 ...
    NO emissions  (time, lat, lon) float32 ...

In [6]:
land_fixed = land.drop_vars('NO emissions') #They are already in the weather dataset
hours = np.arange(0,24,1)
land_fixed = land_fixed.expand_dims({'Hours':hours})
land_fixed = land_fixed.assign_coords(time=land_fixed.time.dt.day)
land_fixed = land_fixed.rename({'time':'Days'})
land_fixed = land_fixed.rename({'lon':'long'})
land_fixed = land_fixed.transpose('lat','long','Days','Hours')  
land_fixed

<xarray.Dataset>
Dimensions:  (Days: 30, Hours: 24, lat: 33, long: 33)
Coordinates:
  * Hours    (Hours) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
  * Days     (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 21 22 23 24 25 26 27 28 29 30
  * long     (long) float64 4.0 4.25 4.5 4.75 5.0 ... 11.0 11.25 11.5 11.75 12.0
  * lat      (lat) float64 43.0 43.25 43.5 43.75 44.0 ... 50.25 50.5 50.75 51.0
Data variables:
    height   (lat, long, Days, Hours) float32 0.0 0.0 0.0 0.0 ... nan nan nan
    built    (lat, long, Days, Hours) float32 0.0 0.0 0.0 0.0 ... nan nan nan

##Weather

In [7]:
weather = xr.open_dataset(fn_weather)
weather

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, latitude: 15, longitude: 15)
Coordinates:
  * latitude           (latitude) float64 44.5 44.6 44.7 44.8 ... 45.7 45.8 45.9
  * longitude          (longitude) float64 8.5 8.6 8.7 8.8 ... 9.6 9.7 9.8 9.9
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/15)
    EMISSIONS_2019     (latitude, longitude, Days, Hours) float32 ...
    u10                (latitude, longitude, Days, Hours) float32 ...
    v10                (latitude, longitude, Days, Hours) float32 ...
    hcc                (latitude, longitude, Days, Hours) float32 ...
    lcc                (latitude, longitude, Days, Hours) float32 ...
    tcc                (latitude, longitude, Days, Hours) float32 ...
    ...                 ...
    tmp                (latitude, longitude, Days, Hours) float32 ...
    sp_hum             (latitude, longitude, Days, Hours) float32 ...
    tcw                (latitude, longitude, Days, Hours) float32 ...
    tot_wind           (latitude, longitude, Days, Hours) float32 ...
    tmp_shift_8        (latitude, longitude, Days, Hours) float32 ...
    tot_wind_shift_12  (latitude, longitude, Days, Hours) float32 ...

In [8]:
#These variables are also in another dataset
weather_fixed = weather.drop_vars('tcw')
weather_fixed = weather_fixed.rename({'longitude':'long'})
weather_fixed = weather_fixed.rename({'latitude':'lat'})
weather_fixed = weather_fixed.transpose('lat','long','Days','Hours')  
weather_fixed

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, lat: 15, long: 15)
Coordinates:
  * lat                (lat) float64 44.5 44.6 44.7 44.8 ... 45.6 45.7 45.8 45.9
  * long               (long) float64 8.5 8.6 8.7 8.8 8.9 ... 9.6 9.7 9.8 9.9
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/14)
    EMISSIONS_2019     (lat, long, Days, Hours) float32 ...
    u10                (lat, long, Days, Hours) float32 ...
    v10                (lat, long, Days, Hours) float32 ...
    hcc                (lat, long, Days, Hours) float32 ...
    lcc                (lat, long, Days, Hours) float32 ...
    tcc                (lat, long, Days, Hours) float32 ...
    ...                 ...
    sp                 (lat, long, Days, Hours) float32 ...
    tmp                (lat, long, Days, Hours) float32 ...
    sp_hum             (lat, long, Days, Hours) float32 ...
    tot_wind           (lat, long, Days, Hours) float32 ...
    tmp_shift_8        (lat, long, Days, Hours) float32 ...
    tot_wind_shift_12  (lat, long, Days, Hours) float32 ...

##Conc

In [9]:
conc = xr.open_dataset(fn_conc)
conc

<xarray.Dataset>
Dimensions:  (Days: 31, Hours: 8, lat: 31, long: 31)
Coordinates:
  * lat      (lat) float64 43.0 43.25 43.5 43.75 44.0 ... 49.75 50.0 50.25 50.5
  * long     (long) float64 4.0 4.25 4.5 4.75 5.0 ... 10.5 10.75 11.0 11.25 11.5
  * Days     (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31
  * Hours    (Hours) int64 0 3 6 9 12 15 18 21
Data variables:
    tcco     (lat, long, Days, Hours) float64 ...
    tcno2    (lat, long, Days, Hours) float64 ...
    tc_no    (lat, long, Days, Hours) float64 ...
    gtco3    (lat, long, Days, Hours) float64 ...
    tcwv     (lat, long, Days, Hours) float64 ...

In [10]:
conc_fidex=conc
hour_bins = np.arange(0,24,1)
conc_fidex = conc_fidex.interp(Hours=hour_bins, method="linear")
conc_fidex = conc_fidex.transpose('lat','long','Days','Hours')  
conc_fidex

<xarray.Dataset>
Dimensions:  (Days: 31, Hours: 24, lat: 31, long: 31)
Coordinates:
  * lat      (lat) float64 43.0 43.25 43.5 43.75 44.0 ... 49.75 50.0 50.25 50.5
  * long     (long) float64 4.0 4.25 4.5 4.75 5.0 ... 10.5 10.75 11.0 11.25 11.5
  * Days     (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31
  * Hours    (Hours) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
Data variables:
    tcco     (lat, long, Days, Hours) float64 0.0009544 0.0009548 ... nan nan
    tcno2    (lat, long, Days, Hours) float64 3.432e-06 3.334e-06 ... nan nan
    tc_no    (lat, long, Days, Hours) float64 9.295e-10 9.842e-10 ... nan nan
    gtco3    (lat, long, Days, Hours) float64 0.007882 0.007857 ... nan nan
    tcwv     (lat, long, Days, Hours) float64 14.48 14.07 13.67 ... nan nan

#Traffic

In [11]:
traffic = xr.open_dataset(fn_traffic)
traffic

<xarray.Dataset>
Dimensions:    (hour: 24, latitude: 60, longitude: 61)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 9.6 9.7 9.8 9.9 10.0
  * latitude   (latitude) float32 43.05 43.15 43.25 43.35 ... 48.75 48.85 48.95
  * hour       (hour) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23
Data variables:
    emissions  (hour, latitude, longitude) float32 ...
    traffic    (hour, latitude, longitude) float32 ...

In [12]:
traffic_fixed=traffic.drop_vars('emissions')
lat_bins = np.arange(43,51.25,0.25)
lon_bins = np.arange(4,12.25,0.25)
traffic_fixed = traffic_fixed.sortby(['latitude','longitude','hour'])  
traffic_fixed = traffic_fixed.interp(latitude=lat_bins, longitude=lon_bins, method="linear")
days = np.arange(1,32,1)
traffic_fixed=traffic_fixed.expand_dims({'Days':days})
traffic_fixed=traffic_fixed.rename({'hour':'Hours'})
traffic_fixed=traffic_fixed.rename({'longitude':'long'})
traffic_fixed=traffic_fixed.rename({'latitude':'lat'})
traffic_fixed=traffic_fixed.transpose('lat','long','Days','Hours')  
traffic_fixed

<xarray.Dataset>
Dimensions:  (Days: 31, Hours: 24, lat: 33, long: 33)
Coordinates:
  * Days     (Days) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31
  * Hours    (Hours) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
  * lat      (lat) float64 43.0 43.25 43.5 43.75 44.0 ... 50.25 50.5 50.75 51.0
  * long     (long) float64 4.0 4.25 4.5 4.75 5.0 ... 11.0 11.25 11.5 11.75 12.0
Data variables:
    traffic  (lat, long, Days, Hours) float64 nan nan nan nan ... nan nan nan

#Merge

In [13]:
tot_dataset = weather_fixed.merge(land_fixed)
tot_dataset = tot_dataset.merge(conc_fidex)
tot_dataset = tot_dataset.merge(traffic_fixed)

tot_dataset

<xarray.Dataset>
Dimensions:            (Days: 31, Hours: 24, lat: 47, long: 47)
Coordinates:
  * lat                (lat) float64 43.0 43.25 43.5 43.75 ... 50.5 50.75 51.0
  * long               (long) float64 4.0 4.25 4.5 4.75 ... 11.5 11.75 12.0
  * Days               (Days) int64 1 2 3 4 5 6 7 8 ... 24 25 26 27 28 29 30 31
  * Hours              (Hours) int64 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Data variables: (12/22)
    EMISSIONS_2019     (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    u10                (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    v10                (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    hcc                (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    lcc                (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    tcc                (lat, long, Days, Hours) float32 nan nan nan ... nan nan
    ...                 ...
    tcco               (lat, long, Days, Hours) float64 0.0009544 ... nan
    tcno2              (lat, long, Days, Hours) float64 3.432e-06 ... nan
    tc_no              (lat, long, Days, Hours) float64 9.295e-10 ... nan
    gtco3              (lat, long, Days, Hours) float64 0.007882 ... nan
    tcwv               (lat, long, Days, Hours) float64 14.48 14.07 ... nan nan
    traffic            (lat, long, Days, Hours) float64 nan nan nan ... nan nan

In [14]:
df = tot_dataset.to_dataframe()
df.head()

EMISSIONS_2019  u10  v10  ...     gtco3       tcwv  traffic
  Hours                                      ...                              
1 0     43.0 4.00             NaN  NaN  NaN  ...  0.007882  14.479529      NaN
             4.25             NaN  NaN  NaN  ...  0.007854  14.662142      NaN
             4.50             NaN  NaN  NaN  ...  0.007826  14.844755      NaN
             4.75             NaN  NaN  NaN  ...  0.007798  15.027369      NaN
             5.00             NaN  NaN  NaN  ...  0.007779  14.954130      NaN

[5 rows x 22 columns]

In [15]:
tot_dataset.to_netcdf('dataset_complete_for_model_CO.nc', 'w', 'NETCDF4')

#Check

In [16]:
df_clean = df.dropna()
df_clean.head()

EMISSIONS_2019       u10  ...       tcwv    traffic
  Hours                                     ...                      
1 12    44.5 8.5    1.937331e-10  1.151261  ...  12.534024   0.000000
  13    44.5 8.5    1.971024e-10  1.192059  ...  12.323180  85.445255
  14    44.5 8.5    1.920485e-10  1.232856  ...  12.112337  84.904842
  15    44.5 8.5    1.954177e-10  1.273654  ...  11.901493  85.746872
  16    44.5 8.5    2.190026e-10  1.036139  ...  11.753887  77.719917

[5 rows x 22 columns]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1643496 entries, (1, 0, 43.0, 4.0) to (31, 23, 51.0, 12.0)
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   EMISSIONS_2019     162000 non-null  float32
 1   u10                166950 non-null  float32
 2   v10                166950 non-null  float32
 3   hcc                166950 non-null  float32
 4   lcc                166950 non-null  float32
 5   tcc                166950 non-null  float32
 6   cvl                166950 non-null  float32
 7   cvh                166950 non-null  float32
 8   sp                 166950 non-null  float32
 9   tmp                166950 non-null  float32
 10  sp_hum             166950 non-null  float32
 11  tot_wind           166950 non-null  float32
 12  tmp_shift_8        166942 non-null  float32
 13  tot_wind_shift_12  166938 non-null  float32
 14  height             760320 non-null  float32
 15  built              7

In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 648 entries, (1, 12, 44.5, 8.5) to (30, 21, 44.5, 8.5)
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EMISSIONS_2019     648 non-null    float32
 1   u10                648 non-null    float32
 2   v10                648 non-null    float32
 3   hcc                648 non-null    float32
 4   lcc                648 non-null    float32
 5   tcc                648 non-null    float32
 6   cvl                648 non-null    float32
 7   cvh                648 non-null    float32
 8   sp                 648 non-null    float32
 9   tmp                648 non-null    float32
 10  sp_hum             648 non-null    float32
 11  tot_wind           648 non-null    float32
 12  tmp_shift_8        648 non-null    float32
 13  tot_wind_shift_12  648 non-null    float32
 14  height             648 non-null    float32
 15  built              648 non-null    float3